# Limpieza de datos (Eliminación de datos faltantes, unión de nuevas columnas, transformación de tipos de datos de columnas, imputación de valores, etc)

**RESUMEN.** Cargamos nuestro dataset con datos desde 2014 hasta 2024, como en la API de la AEMET hay muchas filas con datos faltantes, obtenemos un dataframe con los lugares donde están todos los datos completos y de 3.400.000 datos nos quedamos con un dataset que tiene todas las filas con datos completos (486410), y a este dataset le añadimos las columnas latitud y longitud a cada municipio distinto (149). Como en algunas columnas aparece la palabra 'varias' lo que hacemos es a partir del dataset de 3.400.000 datos obtenemos las medias de los datos en función del día y la provincia en la que estén y aplicamos esas medias como valores en lugar de la palabra 'varias'. Finalmente convertimos y eliminamos algunas columnas.

**Tiempo de ejecución (Ejecutar Todo):**   >1 minuto

In [1]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns', 30)

Cargamos el archivo concatenado con todos los datos (unos 3.400.000)

In [2]:
df_total = pd.read_csv('archivo_concatenado.csv')
df_total.shape

(3398666, 25)

Obtenemos 915 municipios únicos, pero la mayoría de ellos tienen muchos valores NaN, sobretodo en la columna Sol que es el target que queremos obtener

In [3]:
municipios = list(df_total['nombre'].unique())
len(municipios)

915

- Aplicamos una expresión regular para **reemplazar espacios en blanco** con NaN con el método replace de pandas
- Comprobamos los datos nulos del dataset, como se puede observar hay muchos datos faltantes, sobretodo en las últimas 5 columnas, y en especial en nuestra columna target **Sol**

In [4]:
df_total.replace(r'^\s*$', float('nan'), regex=True, inplace=True) 
nulos = df_total.isnull().sum()
nulos

fecha                0
indicativo           0
nombre               0
provincia            0
altitud              0
tmed             73334
prec             97192
tmin             72600
tmax             71983
horatmin         96381
horatmax         94773
hrMedia         201442
hrMax           170672
horaHrMax       172290
hrMin           170496
horaHrMin       172081
dir             710921
velmedia        694873
racha           710837
horaracha       711034
presMax        2563227
horaPresMax    2563258
presMin        2563235
horaPresMin    2563303
sol            2810504
dtype: int64

- Creamos una copia para tener un nuevo dataframe y no hacer cambios en el original.
- Optamos por eliminar las filas con datos faltantes, con lo que obtenemos un total de 486.410 datos prácticamente limpios, conservando sus 25 columnas.
- Ahora tenemos la copia con datos limpios, pero hay varias columnas donde aparece la palabra 'varias', por lo que primero trataremos el dataset original para obtener las medias con más precisión ya que contiene más datos, y después volveremos a esta copia para trabajar con estos datos reales.

In [5]:
df = df_total.copy().dropna()
df.shape

(486410, 25)

In [6]:
nulos_2 = df.isnull().sum()
nulos_2

fecha          0
indicativo     0
nombre         0
provincia      0
altitud        0
tmed           0
prec           0
tmin           0
tmax           0
horatmin       0
horatmax       0
hrMedia        0
hrMax          0
horaHrMax      0
hrMin          0
horaHrMin      0
dir            0
velmedia       0
racha          0
horaracha      0
presMax        0
horaPresMax    0
presMin        0
horaPresMin    0
sol            0
dtype: int64

# Transformaciones en el dataset df_total

- El formato de la fecha es ISO 8601.
- Combinamos la fecha del día con la hora para indicar día y hora exactos en las columnas donde se trabaje con el formato de horas.
- Como el formato es 05:00 (formato de hora), primero comprobamos si la hora es la cadena string 'varias', y si es lo convertimos a NaN, a este valor es al que le vamos a aplicar la media según su provincia.
- Cuando tenemos todas las columnas transformadas, entonces imputamos la media de las provincias y días a la columna donde tenemos las fechas en formato **'%Y-%m-%d %H:%M'**

- Comprobamos que hay 26 horas distintas, de 0 a 24 ambos incluidos, y el string 'Varias', cambiamos las horas que sean 24:00 por 00:00 y luego transformamos Varias a NaT (Not a Time)

In [7]:
horas_unicas_presion_maxima = df_total['horaPresMax'].unique()
horas_unicas_presion_maxima

array([nan, '00', '10', 'Varias', '11', '09', '02', '01', '07', '12',
       '21', '24', '23', '22', '08', '20', '18', '16', '13', '17', '19',
       '15', '03', '05', '14', '04', '06'], dtype=object)

In [8]:
horas_unicas_presion_minima = df_total['horaPresMin'].unique()
horas_unicas_presion_maxima

array([nan, '00', '10', 'Varias', '11', '09', '02', '01', '07', '12',
       '21', '24', '23', '22', '08', '20', '18', '16', '13', '17', '19',
       '15', '03', '05', '14', '04', '06'], dtype=object)

In [9]:
df_total['horaPresMax'] = df_total['horaPresMax'].replace('24', '00')

In [10]:
df_total['horaPresMin'] = df_total['horaPresMin'].replace('24', '00')

- La función **transformar_hora** comprueba si se puede transformar a entero, si se puede transformar entonces le añadimos :00 al final para que se pueda convertir después en pd.to_datetime()

In [11]:
def transformar_hora(hora):
    try:
        hora_int = int(hora)
        if 0 <= hora_int <= 23:
            return str(hora_int) + ':00'
        else:
            return np.nan
    except ValueError:
        return np.nan

In [12]:
df_total['horaPresMax'] = df_total['horaPresMax'].apply(transformar_hora)

In [13]:
df_total['horaPresMin'] = df_total['horaPresMin'].apply(transformar_hora)

- Creamos las nuevas columnas, que unen la fecha de la fila + la hora en esa columna, y el output es en formato datetime. En total convertimos 7 columnas.
- Podemos crear nueva columna o transformar la columna existente, en este caso transformamos los valores de la columna existente.

In [14]:
df_total['horatmax'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horatmax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horatmax'] = df_total['horatmax'].where(df_total['horatmax'].notna() & (df_total['horatmax'] != 'varias'))

In [15]:
df_total['horatmin'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horatmin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horatmin'] = df_total['horatmin'].where(df_total['horatmin'].notna() & (df_total['horatmin'] != 'varias'))

In [16]:
df_total['horaPresMax'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horaPresMax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horaPresMax'] = df_total['horaPresMax'].where(df_total['horaPresMax'].notna() & (df_total['horaPresMax'] != 'varias'))

In [17]:
df_total['horaPresMin'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horaPresMin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horaPresMin'] = df_total['horaPresMin'].where(df_total['horaPresMin'].notna() & (df_total['horaPresMin'] != 'varias'))

In [18]:
df_total['horaHrMax'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horaHrMax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horaHrMax'] = df_total['horaHrMax'].where(df_total['horaHrMax'].notna() & (df_total['horaHrMax'] != 'varias'))

In [19]:
df_total['horaHrMin'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horaHrMin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horaHrMin'] = df_total['horaHrMin'].where(df_total['horaHrMin'].notna() & (df_total['horaHrMin'] != 'varias'))

In [20]:
df_total['horaracha'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_total['horaracha'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_total['horaracha'] = df_total['horaracha'].where(df_total['horaracha'].notna() & (df_total['horaracha'] != 'varias'))

- Agrupamos por fecha y provincia, y obtenemos la media de ese día en esa provincia para imputarsela a los datos faltantes en la columna fecha_horaPresMin. De esta forma de damos al valor faltante un dato del mismo día y provincia, lo cuál teniendo en cuenta que es clima puede tener cierto sentido aunque seguramente variará un poco con respecto a los datos originales. También podríamos haber imputado la mediana en lugar de la media.

In [21]:
df_total['horaPresMin'] = pd.to_datetime(df_total['horaPresMin'], format='%Y-%m-%d %H:%M', errors='coerce')

In [22]:
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horaPresMin'].transform('mean')
df_total['horaPresMin'] = df_total['horaPresMin'].fillna(media_provincias_df_total).round(2)

In [23]:
df_total['horatmax'] = pd.to_datetime(df_total['horatmax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horatmax'].transform('mean')
df_total['horatmax'] = df_total['horatmax'].fillna(media_provincias_df_total).round(2)

df_total['horatmin'] = pd.to_datetime(df_total['horatmin'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horatmin'].transform('mean')
df_total['horatmin'] = df_total['horatmin'].fillna(media_provincias_df_total).round(2)

df_total['horaPresMax'] = pd.to_datetime(df_total['horaPresMax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horaPresMax'].transform('mean')
df_total['horaPresMax'] = df_total['horaPresMax'].fillna(media_provincias_df_total).round(2)

df_total['horaHrMax'] = pd.to_datetime(df_total['horaHrMax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horaHrMax'].transform('mean')
df_total['horaHrMax'] = df_total['horaHrMax'].fillna(media_provincias_df_total).round(2)

df_total['horaHrMin'] = pd.to_datetime(df_total['horaHrMin'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horaHrMin'].transform('mean')
df_total['horaHrMin'] = df_total['horaHrMin'].fillna(media_provincias_df_total).round(2)

df_total['horaracha'] = pd.to_datetime(df_total['horaracha'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_total = df_total.groupby(['fecha', 'provincia'])['horaracha'].transform('mean')
df_total['horaracha'] = df_total['horaracha'].fillna(media_provincias_df_total).round(2)

In [24]:
df_total.head(2)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,horatmin,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,sol
0,2014-01-01,C439J,GÜÍMAR,STA. CRUZ DE TENERIFE,115,"16,0","0,0","12,5","19,5",2014-01-01 09:44:07.741935360,2014-01-01 13:14:39.375000064,NaN,NaN,2014-01-01 16:05:31.304347648,NaN,2014-01-01 12:25:44.444444416,NaN,NaN,NaN,2014-01-01 13:17:31.999999744,NaN,2014-01-01 09:10:00,NaN,2014-01-01 14:20:00,NaN
1,2014-01-01,0255B,SANTA SUSANNA,BARCELONA,40,"7,6","0,0","1,9","13,2",2014-01-01 04:00:00.000000000,2014-01-01 13:10:00.000000000,62.0,89.0,2014-01-01 03:50:00.000000000,47.0,2014-01-01 12:40:00.000000000,NaN,NaN,NaN,2014-01-01 14:37:10.588235264,NaN,2014-01-01 08:30:00,NaN,2014-01-01 03:36:00,NaN


# Volvemos a la copia del dataframe (df) que es nuestro dataset con datos limpios y añadimos las columnas con las coordenadas

In [25]:
municipios = list(df['nombre'].unique())
len(municipios)

149

Ahora tenemos un total de 149 municipios distintos, por lo que decidimos obtener las coordenadas de cada municipio e imputárselas a cada columna según corresponda

- Cargamos el csv con tres columnas llamadas municipio, latitud y longitud. Tiene un total de 149 filas (cada una con un municipio distinto correspondiente a los 149 municipios únicos de nuestro dataset de 486 mil datos)
- Creamos una función **limpiar_nombre** cuyo objetivo es convertir los strings a mayúsculas, y quitar símbolos como espacios, ',', '\', '/' . Esta función se la pasamos a la columna 'municipio' de nuestro dataframe de municipios y a la columna 'nombre' de nuestro dataframe con los datos totales
- Una vez limpias esas columnas, hacemos un merge que itera en las columnas en las que coincide nombre-municipio y le añade al dataframe total las dos columnas latitud y longitud.

In [26]:
# Función para limpiar los nombres (eliminar espacios, comas, comillas y barras)
def limpiar_nombre(nombre):
    return re.sub(r'[ ,"/\\]', '', nombre.strip().lower())

# Cargar los CSV
df_municipios = pd.read_csv('Coordenadas_municipios.csv')

# Limpiar los nombres de los municipios en ambos dataframes
df_municipios['municipio'] = df_municipios['municipio'].apply(limpiar_nombre)
df['nombre'] = df['nombre'].apply(limpiar_nombre)

# Realizar el merge basándonos en los nombres de los municipios
df_resultado = pd.merge(df, df_municipios[['municipio', 'latitud', 'longitud']], left_on='nombre', right_on='municipio', how='left')

# Guardar el resultado en un nuevo CSV
#df_resultado.to_csv('Datos_tiempo_con_coordenadas.csv', index=False)



In [27]:
df_resultado.head(30)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,horatmin,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,sol,municipio,latitud,longitud
0,2014-01-01,3094B,tarancón,CUENCA,808,"7,2","1,2","5,0","9,3",Varias,13:00,92.0,97.0,Varias,86.0,14:30,99.0,"4,4","10,0",Varias,"926,4",00,"921,3",24,"0,0",tarancón,40.010540,-3.006447
1,2014-01-01,B434X,portocolom,ILLES BALEARS,17,"14,1","0,0","10,8","17,4",07:30,14:00,74.0,90.0,Varias,53.0,15:30,99.0,"2,2","9,7",14:50,"1017,0",10,"1014,1",24,"7,8",portocolom,39.415624,3.256301
2,2014-01-01,8293X,xàtiva,VALENCIA,88,"13,0","0,0","7,0","19,0",08:00,13:50,57.0,78.0,08:20,39.0,15:10,28.0,"1,9","11,1",16:40,"1010,4",10,"1005,4",24,"5,2",xàtiva,38.988870,-0.515614
3,2014-01-01,2755X,benavente,ZAMORA,715,"8,0","1,2","6,5","9,5",06:00,15:00,87.0,93.0,Varias,83.0,18:00,22.0,"4,4","13,6",13:20,"934,0",00,"926,3",23,"0,0",benavente,42.003020,-5.674080
4,2014-01-01,C249I,fuerteventuraaeropuerto,LAS PALMAS,25,"15,6","0,0","11,3","20,0",06:20,13:31,66.0,85.0,Varias,46.0,13:18,29.0,"3,1","6,7",07:32,"1022,3",Varias,"1019,6",05,"9,4",fuerteventuraaeropuerto,28.449129,-13.866366
5,2014-01-01,2462,puertodenavacerrada,MADRID,1893,"1,1","15,0","0,2","2,0",Varias,23:55,99.0,100.0,Varias,93.0,06:20,26.0,"5,8","18,1",21:30,"811,6",00,"806,7",23,"0,0",puertodenavacerrada,40.789408,-4.003783
6,2014-01-01,5047E,baza,GRANADA,785,"9,0","0,0","3,5","14,4",07:50,14:20,70.0,82.0,23:50,55.0,13:50,31.0,"2,5","7,2",13:40,"931,5",00,"928,0",15,"6,2",baza,37.490650,-2.774473
7,2014-01-01,4358X,donbenito,BADAJOZ,273,"11,0","0,0","8,4","13,6",08:00,14:30,87.0,100.0,Varias,78.0,Varias,22.0,"2,2","8,3",20:50,"989,9",00,"984,9",20,"0,7",donbenito,38.954300,-5.861752
8,2014-01-01,1387E,acoruñaaeropuerto,A CORUÑA,98,"12,2","4,5","9,7","14,8",00:00,17:06,86.0,95.0,Varias,70.0,21:51,23.0,"8,1","27,2",13:11,"1002,5",00,"990,4",16,"0,0",acoruñaaeropuerto,43.300150,-837883.000000
9,2014-01-01,1212E,asturiasaeropuerto,ASTURIAS,127,"12,4","0,6","7,7","17,0",04:08,Varias,61.0,91.0,Varias,48.0,13:28,23.0,"5,6","27,8",15:45,"998,8",00,"985,5",18,"0,3",asturiasaeropuerto,43.562515,-6.033412


- Volvemos a comprobar que se ha hecho el merge correctamente y que no hay fillas con datos nulos.
- Ahora tenemos dos filas duplicadas (nombre y municipio), las cuáles trataremos más tarde.
- Configuramos el **display** de pandas para poder visualizar el dataset a lo ancho de las columnas.

In [28]:
pd.set_option('display.max_columns', 30)
df_resultado.head(2)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,horatmin,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,sol,municipio,latitud,longitud
0,2014-01-01,3094B,tarancón,CUENCA,808,"7,2","1,2","5,0","9,3",Varias,13:00,92.0,97.0,Varias,86.0,14:30,99.0,"4,4","10,0",Varias,"926,4",00,"921,3",24,"0,0",tarancón,40.010540,-3.006447
1,2014-01-01,B434X,portocolom,ILLES BALEARS,17,"14,1","0,0","10,8","17,4",07:30,14:00,74.0,90.0,Varias,53.0,15:30,99.0,"2,2","9,7",14:50,"1017,0",10,"1014,1",24,"7,8",portocolom,39.415624,3.256301


In [29]:
nulos_3 = df_resultado.isnull().sum()
nulos_3

fecha          0
indicativo     0
nombre         0
provincia      0
altitud        0
tmed           0
prec           0
tmin           0
tmax           0
horatmin       0
horatmax       0
hrMedia        0
hrMax          0
horaHrMax      0
hrMin          0
horaHrMin      0
dir            0
velmedia       0
racha          0
horaracha      0
presMax        0
horaPresMax    0
presMin        0
horaPresMin    0
sol            0
municipio      0
latitud        0
longitud       0
dtype: int64

# Obtenemos un dataset (df_resultado), este es nuestro dataset definitivo, al que le haremos todas las transformaciones necesarias para poder utilizarlo en el análisis descriptivo y el entrenamiento del modelo

- En algunas columnas en lugar de un número tenemos el string **Varias**, que indica que han coincidido varias horas con el mismo dato en el mismo día.
- Cómo no disponemos de más datos, vamos a imputar la media de la hora / o la hora mínima por Provincia, de modo que si una provincia de Cuenca tiene un valor Varias, obtenemos la media o la hora mínima de todos los municipios de la provincia de Cuenca y se lo imputamos a esa fila.

- Iteramos por las columnas del dataframe, y le aplicamos una lambda donde si x es de tipo str y la palabra 'varias' se encuentran dentro de x (transformada a minúscula), se hace una suma de todas las filas que contiene ese valor por columna.

In [30]:
keyword = 'varias'

for col in df_resultado.columns:

    count = df_resultado[col].apply(lambda x: isinstance(x, str) and keyword in x.lower()).sum()
    print(f"Columna '{col}' tiene {count} filas con la palabra '{keyword}'.")

Columna 'fecha' tiene 0 filas con la palabra 'varias'.
Columna 'indicativo' tiene 0 filas con la palabra 'varias'.
Columna 'nombre' tiene 0 filas con la palabra 'varias'.
Columna 'provincia' tiene 0 filas con la palabra 'varias'.
Columna 'altitud' tiene 0 filas con la palabra 'varias'.
Columna 'tmed' tiene 0 filas con la palabra 'varias'.
Columna 'prec' tiene 0 filas con la palabra 'varias'.
Columna 'tmin' tiene 0 filas con la palabra 'varias'.
Columna 'tmax' tiene 0 filas con la palabra 'varias'.
Columna 'horatmin' tiene 23443 filas con la palabra 'varias'.
Columna 'horatmax' tiene 11172 filas con la palabra 'varias'.
Columna 'hrMedia' tiene 0 filas con la palabra 'varias'.
Columna 'hrMax' tiene 0 filas con la palabra 'varias'.
Columna 'horaHrMax' tiene 182287 filas con la palabra 'varias'.
Columna 'hrMin' tiene 0 filas con la palabra 'varias'.
Columna 'horaHrMin' tiene 44586 filas con la palabra 'varias'.
Columna 'dir' tiene 0 filas con la palabra 'varias'.
Columna 'velmedia' tiene 0

In [31]:
df_resultado.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486410 entries, 0 to 486409
Data columns (total 28 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   fecha        486410 non-null  object 
 1   indicativo   486410 non-null  object 
 2   nombre       486410 non-null  object 
 3   provincia    486410 non-null  object 
 4   altitud      486410 non-null  int64  
 5   tmed         486410 non-null  object 
 6   prec         486410 non-null  object 
 7   tmin         486410 non-null  object 
 8   tmax         486410 non-null  object 
 9   horatmin     486410 non-null  object 
 10  horatmax     486410 non-null  object 
 11  hrMedia      486410 non-null  float64
 12  hrMax        486410 non-null  float64
 13  horaHrMax    486410 non-null  object 
 14  hrMin        486410 non-null  float64
 15  horaHrMin    486410 non-null  object 
 16  dir          486410 non-null  float64
 17  velmedia     486410 non-null  object 
 18  racha        486410 non-

- Transformamos las columnas para que aparezcan como Fecha + Hora

In [32]:
df_resultado['horatmax'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horatmax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horatmax'] = df_resultado['horatmax'].where(df_resultado['horatmax'].notna() & (df_resultado['horatmax'] != 'varias'))

In [33]:
df_resultado['horatmin'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horatmin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horatmin'] = df_resultado['horatmin'].where(df_resultado['horatmin'].notna() & (df_resultado['horatmin'] != 'varias'))

In [34]:
df_resultado['horaPresMax'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horaPresMax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horaPresMax'] = df_resultado['horaPresMax'].where(df_resultado['horaPresMax'].notna() & (df_resultado['horaPresMax'] != 'varias'))

In [35]:
df_resultado['horaPresMin'] = pd.to_datetime(df_total['fecha'].astype(str) + ' ' + df_resultado['horaPresMin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horaPresMin'] = df_resultado['horaPresMin'].where(df_resultado['horaPresMin'].notna() & (df_resultado['horaPresMin'] != 'varias'))

In [36]:
df_resultado['horaHrMax'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horaHrMax'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horaHrMax'] = df_resultado['horaHrMax'].where(df_resultado['horaHrMax'].notna() & (df_resultado['horaHrMax'] != 'varias'))

In [37]:
df_resultado['horaHrMin'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horaHrMin'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horaHrMin'] = df_resultado['horaHrMin'].where(df_resultado['horaHrMin'].notna() & (df_resultado['horaHrMin'] != 'varias'))

In [38]:
df_resultado['horaracha'] = pd.to_datetime(df_resultado['fecha'].astype(str) + ' ' + df_resultado['horaracha'].astype(str), format='%Y-%m-%d %H:%M', errors='coerce')
df_resultado['horaracha'] = df_resultado['horaracha'].where(df_resultado['horaracha'].notna() & (df_resultado['horaracha'] != 'varias'))

- Imputamos la media de **los datos del dataset original**

In [39]:
df_resultado['horatmax'] = pd.to_datetime(df_resultado['horatmax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horatmax'].transform('mean')
df_resultado['horatmax'] = df_resultado['horatmax'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horatmin'] = pd.to_datetime(df_resultado['horatmin'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horatmin'].transform('mean')
df_resultado['horatmin'] = df_resultado['horatmin'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horaPresMax'] = pd.to_datetime(df_resultado['horaPresMax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horaPresMax'].transform('mean')
df_resultado['horaPresMax'] = df_resultado['horaPresMax'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horaPresMin'] = pd.to_datetime(df_resultado['horaPresMin'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horaPresMin'].transform('mean')
df_resultado['horaPresMin'] = df_resultado['horaPresMin'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horaHrMax'] = pd.to_datetime(df_resultado['horaHrMax'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horaHrMax'].transform('mean')
df_resultado['horaHrMax'] = df_resultado['horaHrMax'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horaHrMin'] = pd.to_datetime(df_resultado['horaHrMin'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horaHrMin'].transform('mean')
df_resultado['horaHrMin'] = df_resultado['horaHrMin'].fillna(media_provincias_df_resultado).round(2)

df_resultado['horaracha'] = pd.to_datetime(df_resultado['horaracha'], format='%Y-%m-%d %H:%M', errors='coerce')
media_provincias_df_resultado = df_total.groupby(['fecha', 'provincia'])['horaracha'].transform('mean')
df_resultado['horaracha'] = df_resultado['horaracha'].fillna(media_provincias_df_resultado).round(2)



In [40]:
nulos = df_resultado.isnull().sum()
nulos

fecha              0
indicativo         0
nombre             0
provincia          0
altitud            0
tmed               0
prec               0
tmin               0
tmax               0
horatmin          40
horatmax          18
hrMedia            0
hrMax              0
horaHrMax       1292
hrMin              0
horaHrMin        106
dir                0
velmedia           0
racha              0
horaracha         62
presMax            0
horaPresMax    10747
presMin            0
horaPresMin     4236
sol                0
municipio          0
latitud            0
longitud           0
dtype: int64

- Eliminamos datos faltantes y obtenemos un dataframe con 471.228 datos.

In [41]:
df_resultado = df_resultado.dropna()
df_resultado.shape

(471228, 28)

In [42]:
#Obersavamos los datos para comprobar que se aplican los cambios.
df_resultado.tail(10)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,horatmin,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin,sol,municipio,latitud,longitud
486400,2024-11-23,7031X,sanjavieraeropuerto,MURCIA,4,"18,4","0,0","15,6","21,1",2024-11-23 06:58:00.000000000,2024-11-23 12:45:00,79.0,90.0,2015-08-14 12:35:31.764705792,65.0,2024-11-23 14:23:00,2.0,"3,1","6,7",2024-11-23 03:51:00.000000000,"1025,4",2015-08-14 16:48:00.000000000,"1022,7",2015-08-14 15:30:00,"8,7",sanjavieraeropuerto,37.775654,-0.818600
486401,2024-11-23,8096,cuenca,CUENCA,949,"10,2","0,0","6,4","14,0",2024-11-23 00:00:00.000000000,2024-11-23 15:00:00,86.0,96.0,2015-08-14 23:01:00.000000000,68.0,2024-11-23 14:40:00,14.0,"1,4","7,5",2024-11-23 20:20:00.000000000,"915,7",2015-08-14 03:50:00.000000000,"913,6",2015-08-14 10:30:00,"1,8",cuenca,40.071830,-2.134005
486402,2024-11-23,5000C,ceuta,CEUTA,87,"16,8","0,0","14,2","19,5",2024-11-23 05:50:00.000000000,2024-11-23 13:10:00,83.0,91.0,2024-11-23 22:00:00.000000000,76.0,2024-11-23 00:50:00,10.0,"2,8","7,5",2024-11-23 23:50:00.000000000,"1013,0",2015-08-14 04:14:59.999999744,"1010,7",2015-08-14 17:50:00,"7,1",ceuta,35.888290,-5.316195
486403,2024-11-23,5783,sevillaaeropuerto,SEVILLA,34,"17,2","0,0","11,6","22,7",2024-11-23 23:56:00.000000000,2024-11-23 13:53:00,74.0,99.0,2015-08-14 05:41:32.307692288,46.0,2024-11-23 13:56:00,36.0,"1,9","6,1",2024-11-23 17:00:00.000000000,"1019,0",2015-08-14 07:00:00.000000000,"1015,7",2015-08-14 13:46:40,"9,4",sevillaaeropuerto,37.422604,-5.902519
486404,2024-11-23,C430E,izaña,STA. CRUZ DE TENERIFE,2369,"10,0","0,0","7,5","12,5",2024-11-23 06:00:00.000000000,2024-11-23 12:40:00,39.0,53.0,2024-11-23 13:10:00.000000000,27.0,2024-11-23 10:00:00,22.0,"13,1","26,9",2024-11-23 23:40:00.000000000,"770,0",2015-08-14 07:20:00.000000000,"766,0",2015-08-14 04:30:00,"2,5",izaña,41.672555,-2.609148
486405,2024-11-23,1505,lugoaeropuerto,LUGO,442,"14,2","0,0","9,9","18,6",2024-11-23 01:40:00.000000000,2024-11-23 14:50:00,74.0,85.0,2015-08-14 05:41:32.307692288,60.0,2024-11-23 14:40:00,22.0,"3,9","13,6",2024-11-23 14:50:00.000000000,"964,4",2015-08-14 07:00:00.000000000,"957,3",2015-08-14 13:46:40,"1,4",lugoaeropuerto,43.132425,-7.460671
486406,2024-11-23,8019,alicante-elcheaeropuerto,ALICANTE,43,"17,6","0,0","15,3","19,9",2015-08-14 08:17:38.571428608,2024-11-23 12:37:00,81.0,92.0,2024-11-23 19:39:00.000000000,72.0,2024-11-23 05:57:00,2.0,"3,1","7,2",2015-08-14 15:08:04.615384576,"1022,1",2015-08-14 04:30:00.000000000,"1019,8",2015-08-14 15:48:00,"4,5",alicante-elcheaeropuerto,38.285135,-0.562213
486407,2024-11-23,1542,puertodeleitariegos,ASTURIAS,1530,"5,4","0,0","0,9","9,8",2024-11-23 01:00:00.000000000,2024-11-23 23:59:00,83.0,99.0,2024-11-23 01:50:00.000000000,70.0,2024-11-23 23:10:00,13.0,"4,4","19,2",2024-11-23 23:30:00.000000000,"848,8",2015-08-14 13:20:00.000000000,"844,8",2015-08-14 01:40:00,"0,0",puertodeleitariegos,42.999982,-6.419722
486408,2024-11-23,1549,ponferrada,LEON,532,"10,0",Ip,"3,6","16,5",2024-11-23 07:20:00.000000000,2024-11-23 22:40:00,81.0,97.0,2024-11-23 08:10:00.000000000,59.0,2024-11-23 22:40:00,25.0,"1,4","9,7",2024-11-23 22:30:00.000000000,"956,4",2015-08-14 09:00:00.000000000,"950,5",2015-08-14 16:20:00,"4,1",ponferrada,42.546330,-6.590833
486409,2024-11-23,3469A,cáceres,CACERES,394,"14,7","0,0","10,5","18,9",2024-11-23 23:00:00.000000000,2024-11-23 14:00:00,82.0,100.0,2015-08-14 14:49:45.000000000,64.0,2024-11-23 15:30:00,11.0,"2,8","8,1",2024-11-23 13:00:00.000000000,"976,3",2015-08-14 08:00:00.000000000,"973,3",2015-08-14 18:00:00,"8,5",cáceres,39.476180,-6.370760


- Eliminamos la columna **indicativo** ya que nos referiremos al municipio con la columna municipio por lo que ya tiene un "id" único, y también eliminamos la columna **nombre** porque está duplicada con la columna municipio

In [43]:
df_resultado.drop(columns=['indicativo', 'nombre'], inplace=True)

- Convertimos las demás columnas a valores numéricos, categóricas y datatime según corresponda.

In [44]:
df_resultado['sol'] = pd.to_numeric(df_resultado['sol'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['presMin'] = pd.to_numeric(df_resultado['presMin'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['presMax'] = pd.to_numeric(df_resultado['presMax'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['velmedia'] = pd.to_numeric(df_resultado['velmedia'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['racha'] = pd.to_numeric(df_resultado['racha'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['tmed'] = pd.to_numeric(df_resultado['tmed'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['prec'] = pd.to_numeric(df_resultado['prec'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['tmin'] = pd.to_numeric(df_resultado['tmin'].str.replace(',', '.', regex=False), errors='coerce')
df_resultado['tmax'] = pd.to_numeric(df_resultado['tmax'].str.replace(',', '.', regex=False), errors='coerce')

In [45]:
df_resultado['provincia'] = df_resultado['provincia'].astype('category')
df_resultado['municipio'] = df_resultado['municipio'].astype('category')

In [46]:
df_resultado['fecha'] = pd.to_datetime(df_resultado['fecha'], format='%Y-%m-%d')

- Para terminar, volvemos a comprobar los valores que se han quedado como nulos al hacer las conversiones, y observamos que en la columna prec hay algunos string con el valor lp, que significa que ha habido una precipitación intermitente, como no podemos imputar dicho valor eliminamos esas filas.

In [47]:
nulos = df_resultado.isnull().sum()
nulos

fecha              0
provincia          0
altitud            0
tmed               0
prec           15603
tmin               0
tmax               0
horatmin           0
horatmax           0
hrMedia            0
hrMax              0
horaHrMax          0
hrMin              0
horaHrMin          0
dir                0
velmedia           0
racha              0
horaracha          0
presMax            0
horaPresMax        0
presMin            0
horaPresMin        0
sol                0
municipio          0
latitud            0
longitud           0
dtype: int64

# Este es el dataframe definitivo para hacer el análisis descriptivo y entrenar el modelo.
# 440.068 filas y 26 columnas de datos reales completos.

In [48]:
df_resultado = df_resultado.dropna()
df_resultado.shape

(455625, 26)

- Tenemos 4 **dtypes**: category(2), datetime64\[ns](8), float64(15), int64(1)

In [49]:
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 455625 entries, 0 to 486409
Data columns (total 26 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   fecha        455625 non-null  datetime64[ns]
 1   provincia    455625 non-null  category      
 2   altitud      455625 non-null  int64         
 3   tmed         455625 non-null  float64       
 4   prec         455625 non-null  float64       
 5   tmin         455625 non-null  float64       
 6   tmax         455625 non-null  float64       
 7   horatmin     455625 non-null  datetime64[ns]
 8   horatmax     455625 non-null  datetime64[ns]
 9   hrMedia      455625 non-null  float64       
 10  hrMax        455625 non-null  float64       
 11  horaHrMax    455625 non-null  datetime64[ns]
 12  hrMin        455625 non-null  float64       
 13  horaHrMin    455625 non-null  datetime64[ns]
 14  dir          455625 non-null  float64       
 15  velmedia     455625 non-null  float64  

In [53]:
df_resultado.to_csv('datos_clima_limpio.csv', index=False)

In [54]:
df_resultado.shape

(455625, 26)